## 4.4 ニューラルネット

In [1]:
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）
# one-hot encodingされたものを読み込む

train = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed_onehot.csv')

In [2]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [5]:
# tensorflowの警告抑制
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

2023-08-28 17:26:20.431178: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
# -----------------------------------
# ニューラルネットの実装
# -----------------------------------
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

# データのスケーリング
scaler = StandardScaler()
tr_x = scaler.fit_transform(tr_x)
va_x = scaler.transform(va_x)
test_x = scaler.transform(test_x)

# ニューラルネットモデルの構築
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(train_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
batch_size = 128
epochs = 10
history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y))

# バリデーションデータでのスコアの確認
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred, eps=1e-7)
print(f'logloss: {score:.4f}')

# 予測
pred = model.predict(test_x)

Epoch 1/10
59/59 [==============================] - 1s 9ms/step - loss: 0.4457 - accuracy: 0.8187 - val_loss: 0.3874 - val_accuracy: 0.8504
Epoch 2/10
59/59 [==============================] - 0s 7ms/step - loss: 0.3618 - accuracy: 0.8576 - val_loss: 0.3775 - val_accuracy: 0.8480
Epoch 3/10
59/59 [==============================] - 0s 5ms/step - loss: 0.3422 - accuracy: 0.8616 - val_loss: 0.3711 - val_accuracy: 0.8564
Epoch 4/10
59/59 [==============================] - 0s 6ms/step - loss: 0.3216 - accuracy: 0.8729 - val_loss: 0.3638 - val_accuracy: 0.8524
Epoch 5/10
59/59 [==============================] - 0s 6ms/step - loss: 0.3002 - accuracy: 0.8791 - val_loss: 0.3515 - val_accuracy: 0.8596
Epoch 6/10
59/59 [==============================] - 0s 6ms/step - loss: 0.2775 - accuracy: 0.8911 - val_loss: 0.3409 - val_accuracy: 0.8560
Epoch 7/10
59/59 [==============================] - 0s 6ms/step - loss: 0.2491 - accuracy: 0.8991 - val_loss: 0.3240 - val_accuracy: 0.8608
Epoch 8/10
59/59 [==

/home/shion31/dev/kaggle-book/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2845: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(


313/313 [==============================] - 1s 2ms/step


In [7]:
# -----------------------------------
# アーリーストッピング
# -----------------------------------
from keras.callbacks import EarlyStopping

# アーリーストッピングの観察するroundを20とする
# restore_best_weightsを設定することで、最適なエポックでのモデルを使用する
epochs = 50
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(tr_x, tr_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(va_x, va_y), callbacks=[early_stopping])
pred = model.predict(test_x)

Epoch 1/50
59/59 [==============================] - 0s 8ms/step - loss: 0.1741 - accuracy: 0.9285 - val_loss: 0.3008 - val_accuracy: 0.8788
Epoch 2/50
59/59 [==============================] - 0s 6ms/step - loss: 0.1652 - accuracy: 0.9331 - val_loss: 0.3040 - val_accuracy: 0.8780
Epoch 3/50
59/59 [==============================] - 0s 6ms/step - loss: 0.1516 - accuracy: 0.9380 - val_loss: 0.3099 - val_accuracy: 0.8736
Epoch 4/50
59/59 [==============================] - 0s 6ms/step - loss: 0.1450 - accuracy: 0.9405 - val_loss: 0.3110 - val_accuracy: 0.8780
Epoch 5/50
59/59 [==============================] - 0s 6ms/step - loss: 0.1395 - accuracy: 0.9431 - val_loss: 0.3270 - val_accuracy: 0.8792
Epoch 6/50
59/59 [==============================] - 0s 6ms/step - loss: 0.1308 - accuracy: 0.9468 - val_loss: 0.3042 - val_accuracy: 0.8788
Epoch 7/50
59/59 [==============================] - 0s 6ms/step - loss: 0.1212 - accuracy: 0.9501 - val_loss: 0.3162 - val_accuracy: 0.8828
Epoch 8/50
59/59 [==